In [1]:
using LinearAlgebra

include("./MPS.jl")
using .MPSforQuantum
using TensorOperations

## MPOの用意

$Z_1 Z_2 + Z_2 Z_3 + Z_3 Z_4$

$$
\left[\begin{array}{c}
    Z_1 & I_1 & I_1\\
\end{array}\right]
\left[\begin{array}{c}
    Z_2 & 0 & 0\\
    0 & Z_2 & 0\\
    0 & 0 & I_2\\
\end{array}\right]
\left[\begin{array}{c}
    I_3 & 0 & 0\\
    0 & Z_3 & 0\\
    0 & 0 & Z_3\\
\end{array}\right]
\left[\begin{array}{c}
    I_4\\
    I_4\\
    Z_4
\end{array}\right] \quad
$$

In [2]:
pauliZ = convert(Array{ComplexF64,2}, [1 0; 0 -1])
pauliI = convert(Array{ComplexF64,2}, [1 0; 0 1])
zero = convert(Array{ComplexF64,2}, [0 0; 0 0])

O = []
push!(O, dstack((pauliZ, pauliI, pauliI)) )
push!(O, ddstack( [(pauliZ, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliI)] ))
push!(O, ddstack( [(pauliI, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliZ)] ))
push!(O, dstack((pauliI, pauliI, pauliZ)))
O[4]

2×2×3 Array{Complex{Float64},3}:
[:, :, 1] =
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

[:, :, 2] =
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

[:, :, 3] =
 1.0+0.0im   0.0+0.0im
 0.0+0.0im  -1.0+0.0im

In [3]:
N = 4
eps = 1e-3
D = 10

for i in 1:16
C0 = zeros(ComplexF64, 2^N)
C0[i] = 1 # '0001'
#C0 = normalize!(rand(ComplexF64, 2^N))
mps = MPS(C0, D)
# println(restore(mps, 4))
expc = expectation(mps, O)
println(expc)
end

3.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
-3.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
-3.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
3.0 + 0.0im


## 初期状態ベクトルを適当に用意

In [6]:
N = 4
C0 = normalize!(rand(ComplexF64, 2^N))

D = 20
mps = MPS(C0, D, 'r') # convert to MPS
mps_size(mps) # print the size of MPS

expc = expectation(mps, O)
println("Initial Expectation Value: ", expc)

array 1's size: (1, 2)
array 2's size: (2, 4)
array 3's size: (4, 2)
array 4's size: (2, 1)
Num of parameters: 40
2^N: 16
Initial Expectation Value: 0.1432423383228108 - 5.984795992119984e-17im


In [52]:
function R_expression(mps, O, t)
    N_site = size(mps)[1]
    n_phys = 2
    contracted_sites = []
    for i = t+1:N_site
        if i==t+1
            tmp_mps = cat(mps[i][1], mps[i][2], dims=3)
            tmp_mps_dag = cat(mps[i][1]', mps[i][2]', dims=3)
            tmp_O = O[i]
            a_len1 = size(mps[i-1][1])[2]
            a_len2 = size(mps[i][1])[2]
            b_len = size(O[i][1, 1, :, :])[1]
            arr = zeros(ComplexF64, a_len1, a_len2, b_len, b_len, a_len1, a_len2)
            @tensor begin
                arr[a1, a2, b1, b2, a1_, a2_] = tmp_mps_dag[a2, a1, sigma1] * tmp_O[sigma1, sigma2, b1, b2] * tmp_mps[a1_, a2_, sigma2]
            end
            
        elseif i==N_site
            # last step
            tmp_mps = cat(mps[i][1][:], mps[i][2][:], dims=2)
            tmp_mps_dag = cat(mps[i][1]'[:], mps[i][2]'[:], dims=2)
            tmp_O = O[i]
            tmp_site = contracted_sites[i-t-1]
            a_len0 = size(tmp_site)[1]
            a_len1 = size(mps[i-1][1])[2]
            a_len2 = size(mps[i][1])[2]
            b_len = size(O[i][1, 1, :])[1]
            arr_0 = zeros(ComplexF64, n_phys, a_len0, a_len1, b_len, b_len, a_len0)
            arr_1 = zeros(ComplexF64, n_phys, n_phys, a_len0, b_len, b_len, a_len0)
            arr = zeros(ComplexF64, a_len0, b_len, a_len0)
            @tensor begin
                arr_0[sigma2, a0, a1, b0, b1, a0_] = tmp_site[a0, a1, b0, b1, a0_, a1_] * tmp_mps[a1_, sigma2]
                arr_1[sigma1, sigma2, a0, b0, b1, a0_] = tmp_mps_dag[a1, sigma1] * arr_0[sigma2, a0, a1, b0, b1, a0_]
                arr[a0, b0, a0_] = arr_1[sigma1, sigma2, a0, b0, b1, a0_] * tmp_O[sigma1, sigma2, b1]
            end        
        else
            tmp_mps = cat(mps[i][1], mps[i][2], dims=3)
            tmp_mps_dag = cat(mps[i][1]', mps[i][2]', dims=3)
            tmp_O = O[i]
            tmp_site = contracted_sites[i-t-1]
            a_len0 = size(tmp_site)[1]
            a_len1 = size(mps[i-1][1])[2]
            a_len2 = size(mps[i][1])[2]
            b_len = size(O[i][1, 1, :, :])[1]
            arr_0 = zeros(ComplexF64, n_phys, a_len0, a_len1, b_len, b_len, a_len0, a_len2)
            arr_1 = zeros(ComplexF64, n_phys, n_phys, a_len0, a_len2, b_len, b_len, a_len0, a_len2)
            arr = zeros(ComplexF64, a_len0, a_len2, b_len, b_len, a_len0, a_len2)
            @tensor begin
                arr_0[sigma2, a0, a1, b0, b1, a0_, a2_] = tmp_site[a0, a1, b0, b1, a0_, a1_] * tmp_mps[a1_, a2_, sigma2]
                arr_1[sigma1, sigma2, a0, a2, b0, b1, a0_, a2_] = tmp_mps_dag[a2, a1, sigma1] * arr_0[sigma2, a0, a1, b0, b1, a0_, a2_]
                arr[a0, a2, b0, b2, a0_, a2_] = arr_1[sigma1, sigma2, a0, a2, b0, b1, a0_, a2_] * tmp_O[sigma1, sigma2, b1, b2]
            end
        end
        
        push!(contracted_sites, arr)
    end
    return contracted_sites[N_site - t]
end

R_expression (generic function with 1 method)

In [77]:
pauliZ = convert(Array{ComplexF64,2}, [1 0; 0 -1])
pauliI = convert(Array{ComplexF64,2}, [1 0; 0 1])
zero = convert(Array{ComplexF64,2}, [0 0; 0 0])

O = []
push!(O, dstack((pauliZ, pauliI, pauliI)) )
push!(O, ddstack( [(pauliZ, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliI)] ))
push!(O, ddstack( [(pauliI, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliZ)] ))
push!(O, dstack((pauliI, pauliI, pauliZ)))

4-element Array{Any,1}:
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0

### R expression が正しくできているか、期待値計算を使ってチェック

In [94]:
N = 4
D = 20
# C0 = normalize!(rand(ComplexF64, 2^N))

for i in 1:2^N
    C0 = zeros(ComplexF64, 2^N)
    C0[i] = 1 # computational basis
    mps = MPS(C0, D, 'r') # convert to MPS
    #mps_size(mps) # print the size of MPS
    R = R_expression(mps, O, 1)

    tmp_mps_cat = cat(mps[1][1], mps[1][2], dims=3)
    tmp_mps_dag_cat = cat(mps[1][1]', mps[1][2]', dims=3)
    tmp_O = O[1]
    res = zeros(ComplexF64, 1, 1)
    @tensor begin
        res[a0, a0_] = R[a1, b1, a1_] * tmp_mps_cat[a0_, a1_, sigma2] * tmp_O[sigma1, sigma2, b1] * tmp_mps_dag_cat[a1, a0, sigma1]
    end
    println(res, ": ", expectation(MPS(C0, D), O))
    
end

Complex{Float64}[3.0 + 0.0im]: 3.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[-3.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: -3.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[-3.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: -3.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[3.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: 3.0 + 0.0im
